In [ ]:
import pandas as pd
import ipcalc
import parse_functions as pf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Vorbereitung: Einlesen der Daten
- Schwierigkeit: Größe des Datensatzes: 10.365.152 Daten (über 10 Milionen)
- keine eindeutiges Trennzeichen in den Daten vorhanden
- kein von Pandas vorgefertigter Import für Log Dateien

In [ ]:
# Source: https://mmas.github.io/read-apache-access-log-pandas
access_log = pd.read_csv(
    'data/access.log',
    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\]\s)',
    engine='python',
    na_values='-',
    header=None,
    usecols=[0, 3, 4, 5, 6, 7, 8],
    names=['ip', 'time', 'request', 'status', 'size', 'referer', 'user_agent'],
    converters={'time': pf.parse_datetime,
                'request': pf.parse_str,
                'status': int,
                'size': int,
                'referer': pf.parse_str,
                'user_agent': pf.parse_str},
    on_bad_lines='warn')

access_log.head()


## Umwandlung in CSV Datei
- Damit nicht jedes Mal Daten neu über das Log eingelesen werden müssen!
- CSV Datei erstellen und für erneutes einlesen nutzbar (geht schneller)

In [ ]:
# Daten in einer CSV Datei speichern
filename = "data/acces_log.csv"
access_log.to_csv(filename)

In [ ]:
# Erzeuge eine Kopie der Daten als "Backup"
access_log_backup = access_log.copy()
len(access_log_backup)

## Daten aus CSV laden

In [ ]:
# Daten aus CSV laden
filename = "data/acces_log.csv"
access_log_csv = pd.read_csv(filename)
len(access_log_csv)
access_log = access_log_csv

## Daten konvertieren & normalisieren

In [ ]:
# Typen konvertieren
access_log["time"] = pd.to_datetime(access_log["time"])

# Daten normalisieren
access_log['time'] = access_log['time'].dt.tz_convert('UTC')

# Aufgabe 1: Beliebtestes Produkt

> Analysieren Sie welche Produkte beliebt sind. Entwickeln Sie dazu eine Definition eines beliebten Produktes. Stellen Sie die Ergebnisse anschaulich da.

## Definition

> **Unsere Definition:**  
  Das Produkt mit dein meisten Aufrufen auf dem Webserver

## Ergebnis


In [ ]:
most_viewed_products = access_log.loc[access_log['request'].str.contains(r'^GET /product/\d+', na=False)].value_counts(access_log['request'])
print(most_viewed_products[0], most_viewed_products.keys()[0])

# Aufgabe 2

> Untersuchen Sie den Datensatz auf weitere Auffälligkeiten.


## Basisinformationen durch Panda Befehle
- Pandas bietet bereits vordefinierte Befehle um einfache Informationen über die Daten zu ermitteln
- Ein Start mit den Befehlen hilft dabei die Daten zu erkunden
- Speziell bei numerrischen Daten erhält man bereits eine Reihe an spannenden statistischen Informationen, aber auch Daten wie eine solche Log Datei, lohnt es sich beide Befehle kurz anzugucken

### Befehl `.info()`
- Übersicht über die Spalten
- Angabe zu den Datentypen der Spalten

In [ ]:
access_log.info()

### Befehl `.describe()`
- Anzahl der Einträge
- wie viele einzigartige Einträge
- höchste Zahl
- durchschnitt
- einiges mehr

--> Statistische Basisinformationen <br>
--> Parameter `include = 'all'` notwendig, dass auch nicht numerische Daten aufgeführt werden



In [ ]:
access_log.describe(include = 'all')

In [ ]:
access_log.head(5)

In [ ]:
access_log[10000:10200]

ToDo: prüfen ob wir das drin haben wollen / raus nehmen möchten? -> wäre nur ne kleine Hilfestellung zum Zugriff auf Attribute
## Grundlage Zugriff auf Attribute:

- Aufruf möglich durch ``acces_log.ip`` oder ``auch acces_log["ip"]``

In [ ]:
# IP-Adressen mit den meisten Aufrufen 
access_log.value_counts("ip")

In [ ]:
# Vorbereitendes Ausprobieren erkunden --> entfernen wenn es nichts sinvolles mehr ist
print(type(access_log.time[0]))
print("---------------")
print(access_log.time.value_counts())
print("---------------")
print(access_log.time[0])
print(access_log.time[0].date())
print(access_log.time[0].time())
print(access_log.time[0].hour)
print(access_log.time[0].minute)

In [ ]:
# For Schleife zum erstellen der neuen Attribute Tag und Zeit Stunde und Minute
# Das kann ggf. auch schöner gemacht werden. Dauert auf jeden Fall seine Zeit!

#erstellt jeweilige Spalten

"""
i = 0

for timestamp in access_log.time:
    access_log["day"][i] = timestamp.date()
    access_log["time_of_day"][i] = timestamp.time() #ToDo: hier ggf. vorher time zu timestamp machen --> dann 
    access_log["hour"][i] = timestamp.hour
    access_log["minute"][i] = timestamp.minute
    i = i+1

    #print(timestamp, i) """

In [ ]:
# alternative Möglichkeit Spalten festzulegen (get schneller)
days = []
time_of_day = []
hour = []
minute = []


for timestamp in access_log.time:
    days.append(timestamp.date())
    time_of_day.append(timestamp.time()) #ToDo: hier ggf. vorher time zu timestamp machen --> dann 
    hour.append(timestamp.hour)
    minute.append(timestamp.minute)

access_log["day"] = days
access_log["time_of_day"] = time_of_day
access_log["hour"] = hour
access_log["minute"] = minute

In [ ]:
access_log.head()

In [ ]:
# Daten in einer CSV Datei speichern
filename = 'data/acces_log.csv'
access_log.to_csv(filename)

In [ ]:
access_log.value_counts("day")

In [ ]:
access_log.value_counts("time_of_day")

In [ ]:
access_log.value_counts("hour")

## IP-Adressen

In [ ]:
access_log.value_counts("minute")

## Heatmap Tageszeit

In [ ]:
# Wochentag und Tageszeit einlesen
access_log['weekday'] = access_log['time'].dt.weekday
access_log['daytime'] = access_log['time'].dt.hour

daytime_access = access_log.groupby(['daytime']).size().to_frame(name = 'count').reset_index()

fig, ax = plt.subplots(1, 1, figsize = (5, 3))
df_wide = daytime_access.pivot_table(columns='daytime', values='count', aggfunc=lambda x:x)
heatmap = sns.heatmap(df_wide, linewidths=1.0,ax=ax)

ax.set_xlim(0, 23)
ax.set_ylim(0, 1)

heatmap.set_title('Aufrufe nach Tageszeit')
heatmap.set_xlabel('Uhrzeit')


In [ ]:
# Wochentag und Tageszeit einlesen
access_log['weekday'] = access_log['time'].dt.weekday
access_log['daytime'] = access_log['time'].dt.hour

daytime_access = access_log.groupby(['weekday', 'daytime']).size().to_frame(name = 'count').reset_index()

fig, ax = plt.subplots(1, 1, figsize = (5, 3))
df_wide = daytime_access.pivot_table(index='weekday',columns='daytime',values='count', aggfunc=lambda x:x)
heatmap = sns.heatmap(df_wide, linewidths=1.0,ax=ax)

ax.set_xlim(0, 23)
ax.set_ylim(0, 6)
# ax.set_yticks(range(0, 7), ['Mo', 'Di', 'Mi', 'Do', 'Fr', 'Sa', 'So'])

heatmap.set_title('Aufrufe nach Tag/Uhrzeit')
heatmap.set_xlabel('Uhrzeit')
heatmap.set_ylabel('Wochentag')

# 1 => Dienstag


# WIP

In [ ]:
city_ip4 = pd.read_csv('data/geo-city/GeoLite2-City-Blocks-IPv4.csv')
city_ip4.head()

In [ ]:
locations = pd.read_csv('data/geo-city/GeoLite2-City-Locations-de.csv')
locations.head()

In [ ]:
copy = access_log.copy()
copy['ip_n'] = pf.ip_to_int(copy['ip'])
copy.head()

In [ ]:
#print(city_ip4['network'].map(lambda network: ipcalc.Network(network)))

def get_network_borders(network):
    ip_network = ipcalc.Network(network)
    if len(ip_network) > 0:
        return [int(ip_network[0]), int(ip_network[-1])]
    return [0, 0]

# print(get_network_borders('1.0.8.0/21'))

# print(city_ip4['network'][0:5].map(lambda network: get_network_borders(network)))

city_ip4['ip_range'] = city_ip4['network'].map(lambda network: ipcalc.Network(network))
city_ip4['ip_first'] = city_ip4['ip_range'].map(lambda ip: int(ip[0]))
city_ip4['ip_last'] = city_ip4['ip_range'].map(lambda ip: int(ip[-1]))

# city_ip4['ip_first'] = pf.ip_to_int(city_ip4['network'].map(lambda network: ipcalc.Network(network)[0]))
# city_ip4['ip_last'] = pf.ip_to_int(city_ip4['network'].map(lambda network: ipcalc.Network(network)[-1]))

city_ip4.head()

In [ ]:
# ip = 16778240
# filtered = city_ip4.loc[city_ip4['ip_first'].le(ip)].loc[city_ip4['ip_last'].ge(ip)]
# # filtered = filtered.loc[filtered['ip_last'].ge(ip)]
# filtered.head()

# copy['ip_n'][0:5].map(lambda ip: city_ip4.loc[city_ip4['ip_first'].le(ip)].loc[city_ip4['ip_last'].ge(ip)]['network'])

copy['network'] = copy['ip_n'].map(lambda ip: city_ip4.loc[city_ip4['ip_first'].le(ip)].loc[city_ip4['ip_last'].ge(ip)]['network'])
copy.head()

## Open Ideas

- [ ] IP auf Locations mappen
- [x] Nutzung im Tagesverlauf (UTC)
  * Wochentage analysieren
- [ ] Nutzung nach Tageszeit (korrigiert nach Location / Timezone based on IP)
- [ ] Requests außer `GET`?
- [ ] Aufrufe mit Status `!= 200` => Fehler
- [ ] Referers analysieren
- [ ] Nach Nutzer und Pfaden gruppieren und zählen => Entscheidungsfreudigkeit der Nutzer
- [ ] Korrelation untersuchen
- [ ] Sessions von Nutzern zählen / schätzen
- [ ] Browser analysieren